In [9]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
import ConnectionConfig as cc
cc.setupEnvironment()

In [10]:
spark = cc.startLocalCluster("Analyzing") # default 4
spark.getActiveSession()

In [11]:
#EXTRACT
dim_customer = spark.read.format("delta").load("spark-warehouse/dimCustomer")
dim_locks = spark.read.format("delta").load("spark-warehouse/dimLocks")           
fact_rides = spark.read.format("delta").load("spark-warehouse/factRidesS2")

In [12]:
#TRANSFORM
#TEMP VIEW MAKEN  
dim_customer.createOrReplaceTempView("dimCustomer")
dim_locks.createOrReplaceTempView("dimLocks")
fact_rides.createOrReplaceTempView("factRides")

## Wat is de invloed van de woonplaats van de gebruikers op het gebruik van de vehicles?

We kunnen deze vraag beantwoorden door te kijken naar het aantal ritten en de gemiddelde ritafstand, gegroepeerd per woonplaats (stad) van de gebruikers. Dit geeft inzicht in welke steden meer gebruik maken van de fietsen en of de afstanden verschillen per locatie.



In [13]:
influence_residence = spark.sql("""
SELECT 
    dc.city AS woonplaats,
    COUNT(fr.rideId) AS totaal_aantal_ritten,
    AVG(fr.rideDistance) AS gemiddelde_ritafstand,
    COUNT(DISTINCT fr.customerSK) AS unieke_gebruikers,
    COUNT(fr.rideId) / COUNT(DISTINCT fr.customerSK) AS ritten_per_gebruiker
FROM 
    factRides AS fr
JOIN 
    dimCustomer AS dc ON fr.customerSK = dc.customer_SK
GROUP BY 
    dc.city
ORDER BY 
    totaal_aantal_ritten DESC;

     """)

influence_residence.show()


+--------------------+--------------------+---------------------+-----------------+--------------------+
|          woonplaats|totaal_aantal_ritten|gemiddelde_ritafstand|unieke_gebruikers|ritten_per_gebruiker|
+--------------------+--------------------+---------------------+-----------------+--------------------+
|           Antwerpen|              724938| 2.585581393161493...|             8978|   80.74604588995322|
|Beveren-Waas/Haas...|              179879| 2.782246337415048...|             2218|   81.09963931469792|
|             Schoten|              177412| 2.779640726644457...|             2168|   81.83210332103322|
| Wilrijk (Antwerpen)|              164747| 3.180984006358998...|             2032|   81.07627952755905|
|  Deurne (Antwerpen)|              152051| 2.975576149627153...|             1834|   82.90676117775354|
|          Brasschaat|              131064| 2.779454845159585...|             1628|   80.50614250614251|
|    Kapellen (Antw.)|              131058| 2.779288126

## We willen voorspellen welke sloten preventief onderhoud nodig hebben. Bekijk hoe vaak slotnummers relatief gezien gebruikt worden.


Om sloten te identificeren die mogelijk preventief onderhoud nodig hebben, kijken we naar hoe vaak elk slot wordt gebruikt, zowel als start- als eindslot.

In [14]:
locks_maintenance =  spark.sql(""" 
SELECT 
    dl.stationnr AS stationnummer,
    dl.stationlocknr AS slotnummer,
    COUNT(fr.rideId) AS gebruiksfrequentie,
    RANK() OVER (ORDER BY COUNT(fr.rideId) DESC) AS gebruiks_ranking
FROM 
    factRides AS fr
JOIN 
    dimLocks AS dl ON fr.startLockSK = dl.lockSK OR fr.endLockSK = dl.lockSK
GROUP BY 
    dl.stationnr, dl.stationlocknr
ORDER BY 
    gebruiksfrequentie DESC;
    """)
locks_maintenance.show()


+-------------+----------+------------------+----------------+
|stationnummer|slotnummer|gebruiksfrequentie|gebruiks_ranking|
+-------------+----------+------------------+----------------+
|          103|        18|             22675|               1|
|          145|        18|             22442|               2|
|          155|        36|             22414|               3|
|          062|        18|             22365|               4|
|          115|        18|             22343|               5|
|          102|        18|             22282|               6|
|          052|        18|             22276|               7|
|          053|        18|             22224|               8|
|          228|        36|             22220|               9|
|          124|        18|             22215|              10|
|          076|        18|             22209|              11|
|          064|        18|             22187|              12|
|          033|        35|             22176|          

## Als een klant zijn abonnement stopzet, willen we kunnen voorspellen op welke stations dit het meeste effect zal hebben.

Hier analyseren we het gebruik van stations door klanten die hun abonnement hebben stopgezet, zodat we kunnen zien op welke stations het effect van opzeggingen het grootst is.

In [15]:
subscription_effect= spark.sql("""
SELECT 
    dl.stationnr AS stationnummer,
    COUNT(fr.rideId) AS aantal_ritten_door_stopgezette_klanten
FROM 
    factRides AS fr
JOIN 
    dimLocks AS dl ON fr.startLockSK = dl.lockSK OR fr.endLockSK = dl.lockSK
JOIN 
    dimCustomer AS dc ON fr.customerSK = dc.customer_SK
WHERE 
    dc.scd_end IS NOT NULL -- Alleen klanten die hun abonnement hebben stopgezet
GROUP BY 
    dl.stationnr
ORDER BY 
    aantal_ritten_door_stopgezette_klanten DESC; 
    """)
subscription_effect.show()

+-------------+--------------------------------------+
|stationnummer|aantal_ritten_door_stopgezette_klanten|
+-------------+--------------------------------------+
|          227|                                 78837|
|          305|                                 78625|
|          225|                                 78383|
|          307|                                 76385|
|          033|                                 75287|
|          123|                                 73418|
|          043|                                 73176|
|          059|                                 73144|
|          061|                                 72648|
|          203|                                 72324|
|          137|                                 72251|
|          055|                                 72191|
|          050|                                 71902|
|          127|                                 71891|
|          105|                                 71845|
|         

## Welke klanten maken de meeste en langste afstand?

We willen weten welke klanten het langst afstand maken.



In [16]:
langste_rides = spark.sql("""
SELECT 
    dc.customer_SK AS klant_id,
    COUNT(fr.rideId) AS totaal_aantal_ritten,
    SUM(fr.rideDistance) AS totale_afgelegde_afstand,
    AVG(fr.rideDistance) AS gemiddelde_afstand_per_rit
FROM 
    factRides AS fr
JOIN 
    dimCustomer AS dc ON fr.customerSK = dc.customer_SK
GROUP BY 
    dc.customer_SK
ORDER BY 
    totaal_aantal_ritten DESC, gemiddelde_afstand_per_rit DESC
LIMIT 10;
""")
langste_rides.show()

+----------+--------------------+------------------------+--------------------------+
|  klant_id|totaal_aantal_ritten|totale_afgelegde_afstand|gemiddelde_afstand_per_rit|
+----------+--------------------+------------------------+--------------------------+
|8589938715|                 339|    1004.698159272662...|      2.963711384285140...|
|8589975817|                 334|    786.0842563966931...|      2.353545677834410...|
|8589975818|                 334|    786.0842563966931...|      2.353545677834410...|
|8589985963|                 332|    837.9135094882245...|      2.523835871952483...|
|8589993524|                 330|    920.0489266057120...|      2.788027050320339...|
|8589983559|                 324|    862.5686670843245...|      2.662248972482483...|
|8589991978|                 322|    856.7873982340437...|      2.660830429298272...|
|8589957643|                 321|    945.8726946462570...|      2.946643908555317...|
|8589971261|                 321|    886.6543309809647

### Welke abonnementsvormen leiden tot het hoogste aantal ritten en de langste ritafstanden per klant?


In [17]:
analyse_stations = spark.sql("""
SELECT 
    dc.subscription_type AS abonnementsvorm,
    COUNT(fr.rideId) AS totaal_aantal_ritten,
    AVG(fr.rideDistance) AS gemiddelde_ritafstand,
    COUNT(DISTINCT fr.customerSK) AS unieke_gebruikers,
    COUNT(fr.rideId) / COUNT(DISTINCT fr.customerSK) AS ritten_per_gebruiker
FROM 
    factRides AS fr
JOIN 
    dimCustomer AS dc ON fr.customerSK = dc.customer_SK
GROUP BY 
    dc.subscription_type
ORDER BY 
    totaal_aantal_ritten DESC;

    """)
analyse_stations.show()


+---------------+--------------------+---------------------+-----------------+--------------------+
|abonnementsvorm|totaal_aantal_ritten|gemiddelde_ritafstand|unieke_gebruikers|ritten_per_gebruiker|
+---------------+--------------------+---------------------+-----------------+--------------------+
|           JAAR|             3365724| 2.779220263972375...|            36788|   91.48972491029683|
|          MAAND|              142242| 2.792381524787680...|             5499|  25.866884888161483|
|            DAG|              126902| 2.807425254426819...|             2505|   50.65948103792415|
+---------------+--------------------+---------------------+-----------------+--------------------+



In [18]:
spark.stop()